# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("05-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'12|91\n25|68\n25|41\n41|34\n41|45\n41|24\n55|32\n55|19\n55|12\n55|85\n32|89\n32|96\n32|19\n32|12\n32|81\n19|81\n19|89\n19|76\n19|38\n19|68\n19|23\n87|73\n87|91\n87|38\n87|37\n87|82\n87|89\n87|29\n89|62\n89|15\n89|76\n89|95\n89|73\n89|68\n89|35\n89|23\n35|22\n35|71\n35|55\n35|85\n35|76\n35|73\n35|94\n35|97\n35|83\n18|22\n18|55\n18|32\n18|68\n18|41\n18|93\n18|37\n18|53\n18|85\n18|83\n37|32\n37|83\n37|85\n37|22\n37|53\n37|16\n37|23\n37|97\n37|55\n37|93\n37|24\n52|58\n52|25\n52|96\n52|42\n52|28\n52|49\n52|91\n52|12\n52|31\n52|45\n52|38\n52|19\n85|91\n85|52\n85|24\n85|14\n85|13\n85|32\n85|28\n85|12\n85|31\n85|42\n85|58\n85|81\n85|49\n84|87\n84|45\n84|15\n84|52\n84|42\n84|58\n84|12\n84|71\n84|19\n84|85\n84|53\n84|81\n84|91\n84|96\n58|82\n58|57\n58|25\n58|91\n58|49\n58|15\n58|89\n58|38\n58|73\n58|42\n58|45\n58|28\n58|14\n58|19\n58|81\n16|32\n16|13\n16|29\n16|87\n16|85\n16|42\n16|89\n16|19\n16|97\n16|71\n16|12\n16|24\n16|28\n16|54\n16|52\n16|31\n81|95\n81|15\n81|49\n81|25\n81|14\n81|78\n81|35

In [4]:
test_data_raw = r"""47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""



def preprocess_data (data):
    # dtype='U10'
    data_trans = data.split("\n\n")
    orderings = [(int(row.split("|")[0]), int(row.split("|")[1]))  for row in  data_trans[0].split("\n")]
    updates = [[int(num) for num in  row.split(",")] for row in  data_trans[1].split("\n")]
    # for row in data.split("\n"):   
    #     reports.append(np.array([int(item) for item in row.split(" ")]))

    return orderings, updates
test_data = preprocess_data(test_data_raw)
# test_data2 = preprocess_data(test_data_raw2)
display(test_data)

([(47, 53),
  (97, 13),
  (97, 61),
  (97, 47),
  (75, 29),
  (61, 13),
  (75, 53),
  (29, 13),
  (97, 29),
  (53, 29),
  (61, 53),
  (97, 53),
  (61, 29),
  (47, 13),
  (75, 47),
  (97, 75),
  (47, 61),
  (75, 61),
  (47, 29),
  (75, 13),
  (53, 13)],
 [[75, 47, 61, 53, 29],
  [97, 61, 53, 29, 13],
  [75, 29, 13],
  [75, 97, 47, 61, 53],
  [61, 13, 29],
  [97, 13, 75, 29, 47]])

In [5]:
data = preprocess_data(data_raw)
data

([(12, 91),
  (25, 68),
  (25, 41),
  (41, 34),
  (41, 45),
  (41, 24),
  (55, 32),
  (55, 19),
  (55, 12),
  (55, 85),
  (32, 89),
  (32, 96),
  (32, 19),
  (32, 12),
  (32, 81),
  (19, 81),
  (19, 89),
  (19, 76),
  (19, 38),
  (19, 68),
  (19, 23),
  (87, 73),
  (87, 91),
  (87, 38),
  (87, 37),
  (87, 82),
  (87, 89),
  (87, 29),
  (89, 62),
  (89, 15),
  (89, 76),
  (89, 95),
  (89, 73),
  (89, 68),
  (89, 35),
  (89, 23),
  (35, 22),
  (35, 71),
  (35, 55),
  (35, 85),
  (35, 76),
  (35, 73),
  (35, 94),
  (35, 97),
  (35, 83),
  (18, 22),
  (18, 55),
  (18, 32),
  (18, 68),
  (18, 41),
  (18, 93),
  (18, 37),
  (18, 53),
  (18, 85),
  (18, 83),
  (37, 32),
  (37, 83),
  (37, 85),
  (37, 22),
  (37, 53),
  (37, 16),
  (37, 23),
  (37, 97),
  (37, 55),
  (37, 93),
  (37, 24),
  (52, 58),
  (52, 25),
  (52, 96),
  (52, 42),
  (52, 28),
  (52, 49),
  (52, 91),
  (52, 12),
  (52, 31),
  (52, 45),
  (52, 38),
  (52, 19),
  (85, 91),
  (85, 52),
  (85, 24),
  (85, 14),
  (85, 13),
  (8

In [6]:
def solution (data, verbose=False):

    def middle_number (ordering):
        index = len(ordering) // 2
        return ordering[index]

    orderings, updates = data[0], data[1]

    valid_updates = []
    for update in updates:
        update_set = set(update)
        
        related_orderings = [ordering for ordering in orderings if ordering[0] in update_set and ordering[1] in update_set]
        
        update_orderings = {update[i]: set(update[i+1:])  for i in range(len(update)-1)}
        update_orderings[update[-1]] = set([])

        valid = True
        for ordering in related_orderings:
            if ordering[0] in  update_orderings[ordering[1]]:
                valid = False
                break
        
        if valid:
            valid_updates.append(update)
            

    
    
    total = sum([middle_number(update)  for update in valid_updates])
    return total




sol = solution(test_data, verbose=True)

print(sol)


143


In [7]:
sol = solution(data)
print(sol)

3608


# Part 2

In [ ]:
def solution2 (data, verbose=False):


    def middle_number (ordering):
        index = len(ordering) // 2
        return ordering[index]

    orderings, updates = data[0], data[1]

    valid_updates = []
    
    
        
    for update in updates:
        update_set = set(update)
        
        related_orderings = [ordering for ordering in orderings if ordering[0] in update_set and ordering[1] in update_set]
        
        order_dict = {}
    
        for ordering in related_orderings:
            if ordering[0] not in order_dict:
                order_dict[ordering[0]] = [ordering[1]]
            else:
                order_dict[ordering[0]] = order_dict[ordering[0]] + [ordering[1]]
            
        reversed_order_dict = {len(val):key for key, val in order_dict.items()}
        full_ordering = [reversed_order_dict[i] for i in range(len(reversed_order_dict.keys()), 0, -1)]
        full_ordering.append(order_dict[full_ordering[-1]][0])
        
        if update != full_ordering:
            valid_updates.append(full_ordering)

    
    
    total = sum([middle_number(update)  for update in valid_updates])
    return total




sol = solution2(test_data, verbose=True)
print(sol)
# print(dists)

# display(sum(sol))

    

123


In [39]:
sol = solution2(data)
print(sol)

4922
